In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# # 가장 어큐러시 높은 모델 확인

# import os
# import re

# # 모델 저장 경로
# model_save_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/PARK'

# # 체크포인트 파일들의 리스트
# checkpoint_files = os.listdir(model_save_path)

# # 정확도를 저장할 딕셔너리
# accuracy_dict = {}

# # 각 파일명에서 정확도 추출
# for file in checkpoint_files:
#     # 파일명에서 정확도 값을 추출하는 정규 표현식
#     match = re.search(r"(\d+\.\d+)\.hdf5$", file)
#     if match:
#         accuracy = float(match.group(1))
#         accuracy_dict[file] = accuracy

# # 가장 높은 정확도를 가진 모델 찾기
# best_model_file = max(accuracy_dict, key=accuracy_dict.get)
# best_accuracy = accuracy_dict[best_model_file]

# print(f"Best Model: {best_model_file}")
# print(f"Accuracy: {best_accuracy}")

In [6]:
# import os
# import re

# # 모델 저장 경로
# model_save_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/PARK'

# # 체크포인트 파일들의 리스트
# checkpoint_files = os.listdir(model_save_path)

# # 정확도를 저장할 딕셔너리
# accuracy_dict = {}

# # 각 파일명에서 정확도 추출
# for file in checkpoint_files:
#     # 파일명에서 정확도 값을 추출하는 정규 표현식
#     match = re.search(r"(\d+\.\d+)\.hdf5$", file)
#     if match:
#         accuracy = float(match.group(1))
#         accuracy_dict[file] = accuracy

# # 가장 높은 정확도를 가진 모델 찾기
# best_model_file = None
# best_accuracy = None

# if accuracy_dict:
#     best_model_file = max(accuracy_dict, key=accuracy_dict.get)
#     best_accuracy = accuracy_dict[best_model_file]
#     print(f"Best Model: {best_model_file}")
#     print(f"Accuracy: {best_accuracy}")
# else:
#     print("No models found in the directory.")


No models found in the directory.


In [ ]:
# import os
# from tensorflow.keras.models import load_model
# model_save_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/PARK'

# # 모델 불러오기
# model = load_model(os.path.join(model_save_path, best_model_file))

In [10]:
import os
from tensorflow.keras.models import load_model
model_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/PARK/resnet_model_sigmoid_relu.h5의 사본'

# 모델 불러오기
model = load_model(model_path)

In [13]:
# 정답, 오답 이미지 분류해서 시각화

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터셋 경로
dataset_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/data/current_dataset'

# 이미지 크기 및 배치 크기 설정
img_width, img_height = 224, 224  # ResNet50의 기본 이미지 크기
batch_size = 128

# 이미지 데이터 생성기
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# 훈련 및 검증 데이터 생성기
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)


import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# TensorFlow 로깅 레벨 설정 (중간 메시지 숨김)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# 테스트 데이터셋 경로와 모델 경로 설정
test_dataset_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/data/test_dataset'
model_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/PARK/resnet_model_sigmoid_relu.h5의 사본'

# 모델 불러오기
model = load_model(model_path)


# 테스트 데이터셋에 대한 ImageDataGenerator 생성
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

# 클래스 인덱스 가져오기
class_indices = test_generator.class_indices
class_names = list(class_indices.keys())

# 모든 예측을 수행하고 결과를 저장
correct_predictions = {}
incorrect_predictions = {}
for _ in range(len(test_generator)):
    x, y = next(test_generator)
    image = x[0]  # 이미지 데이터
    true_label = class_names[np.argmax(y[0])]
    prediction = model.predict(x, verbose=0)
    predicted_class = class_names[np.argmax(prediction)]

    # 여기서 image 대신 x[0]을 저장합니다.
    if true_label == predicted_class:
        if true_label not in correct_predictions:
            correct_predictions[true_label] = []
        correct_predictions[true_label].append((image, predicted_class))  # 이미지 데이터 저장
    else:
        if true_label not in incorrect_predictions:
            incorrect_predictions[true_label] = []
        incorrect_predictions[true_label].append((image, predicted_class))  # 이미지 데이터 저장

# 시각화 함수 정의
def visualize_predictions(predictions, title):
    num_images = sum(len(v) for v in predictions.values())
    plt.figure(figsize=(10, num_images // 4 * 4))  # 가로 10인치, 세로는 이미지 수에 비례하게 조정

    i = 1
    for class_name, images in predictions.items():
        for img, pred_class in images:
            plt.subplot(num_images // 4 + 1, 4, i)
            plt.imshow(img)
            if class_name != pred_class:
                plt.title(f"{class_name} as {pred_class}", size=14, color='red')
                plt.gca().add_patch(plt.Rectangle((0, 0), 223, 223, fill=False, edgecolor='red', lw=2))
            else:
                plt.title(f"Correct: {class_name}", size=14)
            plt.axis('off')
            i += 1
    plt.suptitle(title, size=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # 상단 제목에 여백을 줍니다.
    plt.show()

# 맞춘 예측 시각화
visualize_predictions(correct_predictions, "correct_predictions")

# 틀린 예측 시각화
visualize_predictions(incorrect_predictions, "incorrect_predictions")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 각 카테고리별 정확도 출력

from sklearn.metrics import accuracy_score

# 전체 정확도를 위한 리스트
y_true = []
y_pred = []

# 각 카테고리별 정확도를 위한 딕셔너리
category_true = {category: [] for category in class_names}
category_pred = {category: [] for category in class_names}

# 이미지 데이터를 배치 단위로 불러오기
for _ in range(len(test_generator)):
    x, y = next(test_generator)
    true_category = class_names[np.argmax(y[0])]
    y_true.append(true_category)

    # 모델 예측
    prediction = model.predict(x, verbose=0)
    predicted_category = class_names[np.argmax(prediction)]
    y_pred.append(predicted_category)

    # 카테고리별 리스트에 추가
    category_true[true_category].append(true_category)
    category_pred[true_category].append(predicted_category)

# 전체 정확도 계산
total_accuracy = accuracy_score(y_true, y_pred)

# 각 카테고리별 정확도 계산
category_accuracy = {}
for category in class_names:
    category_accuracy[category] = accuracy_score(category_true[category], category_pred[category])

# 결과 출력
print(f"Total Accuracy: {total_accuracy}")
for category, acc in category_accuracy.items():
    print(f"Accuracy for {category}: {acc}")